
# Wittrick-Williams算法的高级Python实现示例

包含完整的**动力刚度矩阵计算**和更详细的测试用例:
##### 这个示例展示了如何在实际工程中应用Wittrick-Williams算法


In [ ]:
import numpy as np
import math
from typing import List, Tuple, Optional
from dataclasses import dataclass
import matplotlib.pyplot as plt

#### 定义梁单元类型

In [ ]:
@dataclass
class BeamElement:
    """梁单元类型定义（更完整版本）"""
    element_id: int
    node_i: int           # i结点编号
    node_j: int           # j结点编号
    length: float         # 单元长度
    EA: float            # 轴向刚度
    EI: float            # 弯曲刚度
    mass_per_length: float  # 单位长度质量
    cos_alpha: float = 0.0  # 方向余弦
    sin_alpha: float = 0.0  # 方向正弦

    def __post_init__(self):
        """计算派生属性"""
        self.mass_total = self.mass_per_length * self.length  #梁单元重量

#### 定义“动力刚度矩阵计算类”，获得梁单元的局部坐标系下的刚度矩阵

In [ ]:
class DynamicStiffnessMatrix:
    """动力刚度矩阵计算类"""

    @staticmethod
    def element_dynamic_stiffness(element: BeamElement, omega: float) -> np.ndarray:
        """
        计算单个梁单元的动力刚度矩阵

        Args:
            element: 梁单元
            omega: 圆频率 (rad/s)

        Returns:
            6x6动力刚度矩阵
        """
        L = element.length
        EA = element.EA
        EI = element.EI
        m = element.mass_per_length   # 单位长度质量

        # 无量纲频率参数
        omega_squared = omega ** 2
        alpha_squared = omega_squared * m * L ** 4 / EI  # 弯曲频率参数
        beta_squared = omega_squared * m * L ** 2 / EA   # 轴向频率参数

        # 动力刚度矩阵系数
        if alpha_squared > 1e-10:  # 避免数值问题
            alpha = math.sqrt(alpha_squared)
            s = math.sin(alpha)
            c = math.cos(alpha)
            sh = math.sinh(alpha)
            ch = math.cosh(alpha)

            # 弯曲动力刚度系数
            denom = 2 * (1 - c * ch)
            if abs(denom) < 1e-12:
                # 接近共振频率时的处理
                k11 = k22 = EI / L ** 3 * 12
                k12 = k21 = EI / L ** 2 * 6
            else:
                k11 = k22 = EI / L ** 3 * alpha ** 4 * (s * sh) / denom
                k12 = k21 = EI / L ** 2 * alpha ** 2 * (s * ch - c * sh) / denom
        else:
            # 静力刚度（低频近似）
            k11 = k22 = 12 * EI / L ** 3
            k12 = k21 = 6 * EI / L ** 2

        # 轴向动力刚度
        if beta_squared > 1e-10:
            beta = math.sqrt(beta_squared)
            k_axial = EA / L * beta / math.tan(beta)
        else:
            k_axial = EA / L

        # 组装6x6动力刚度矩阵（局部坐标系）
        K_local = np.zeros((6, 6))

        # 轴向刚度
        K_local[0, 0] = K_local[3, 3] = k_axial
        K_local[0, 3] = K_local[3, 0] = -k_axial

        # 弯曲刚度
        K_local[1, 1] = K_local[4, 4] = k11
        K_local[1, 4] = K_local[4, 1] = -k11
        K_local[2, 2] = K_local[5, 5] = k22
        K_local[2, 5] = K_local[5, 2] = k12
        K_local[1, 2] = K_local[2, 1] = k12
        K_local[4, 5] = K_local[5, 4] = k21
        K_local[1, 5] = K_local[5, 1] = -k12
        K_local[2, 4] = K_local[4, 2] = -k21

        return K_local

#### 计算坐标转换矩阵

In [ ]:
    @staticmethod  #静态方法装饰器，不需要访问实例属性或方法， 可以直接通过类名调用， 无需创建实例
    def transformation_matrix(cos_alpha: float, sin_alpha: float) -> np.ndarray:
        """
        计算坐标转换矩阵

        Args:
            cos_alpha: 方向余弦
            sin_alpha: 方向正弦

        Returns:
            6x6转换矩阵
        """
        T = np.zeros((6, 6))

        # 2x2转换子矩阵
        T_sub = np.array([[cos_alpha, sin_alpha],
                         [-sin_alpha, cos_alpha]])

        # 填充6x6矩阵
        T[0:2, 0:2] = T_sub
        T[2, 2] = 1.0
        T[3:5, 3:5] = T_sub
        T[5, 5] = 1.0

        return T